# README

This notebook gives an example of how to run the GEM-PRO pipeline on a **SBML model**.

## Installation

See: https://github.com/nmih/ssbio/blob/master/README.md
- If something isn't working, make sure to update the repository before you do anything (git pull)

## Quick start

I just want to get structures ASAP! How can I do that?

-- TODO

In [1]:
# Import the GEM-PRO class
from ssbio.pipeline.gempro import GEMPRO

In [2]:
# Other imports for this example
import os
import pandas as pd
import os.path as op

In [16]:
# Printing multiple outputs per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
# Create logger
import logging
logger = logging.getLogger()

############# SET YOUR LOGGING LEVEL HERE #############
# - CRITICAL
#     - Only really important messages shown
# - ERROR
#     - Major errors
# - WARNING
#     - Warnings that don't affect running of the pipeline
# - INFO
#     - Info such as the number of structures mapped per gene
# - DEBUG
#     - Really detailed information that will print out a lot of stuff
logger.setLevel(logging.INFO)
#######################################################

## Initialization of the project

Set these three things:

- GEM_NAME
    - Your project name
- ROOT_DIR
    - The directory where the GEM_NAME folder will be created
- GENES
    - Your list of gene IDs
    
A directory will be created in ROOT_DIR named after your GEM_NAME. The folders are:
```
    .
    ├── data  # where dataframes are stored
    ├── figures  # where figures are stored
    ├── model  # where SBML and GEM-PRO models are stored
    ├── notebooks  # location of any ipython notebooks for analyses
    ├── sequences  # sequences are stored here, in gene specific folders
    │   ├── <gene_id1>
    │   │   └── sequence.fasta
    │   ├── <gene_id2>
    │   └── ...
    └── structures
        ├── by_gene  # structures are stored here, in gene specific folders
        │   ├── <gene_id1>
        │   │   └── 1abc.pdb
        │   ├── <gene_id2>
        │   └── ...
        └── by_complex  # complexes for reactions are stored here (in progress)
```

In [4]:
GEM_NAME = 'iNJ661'

ROOT_DIR = '/home/nathan/projects/'

GEM_FILE = '/home/nathan/projects/iNJ661/model/iNJ661.json'

In [5]:
# Create the GEM-PRO project
# Specify that the model is in json format
my_gempro = GEMPRO(gem_name=GEM_NAME, root_dir=ROOT_DIR, gem_file_path=GEM_FILE, gem_file_type='json')

INFO:ssbio.pipeline.gempro:Loaded model: /home/nathan/projects/iNJ661/model/iNJ661.json
INFO:ssbio.pipeline.gempro:Number of reactions: 1025
INFO:ssbio.pipeline.gempro:Number of reactions linked to a gene: 720
INFO:ssbio.pipeline.gempro:Number of genes (excluding spontaneous): 661
INFO:ssbio.pipeline.gempro:Number of metabolites: 826


## Mapping gene ID to sequence

**First, we want to set a sequence for each of the genes in our model.** This can happen one of two ways:

1. I want to map the gene IDs in the model (I don't have the original genome sequence)
2. I have the sequences for my organism

---------------------

### 1. I want to map the gene IDs in the model


##### Mapping with KEGG

- Files downloaded:
    - Per gene, the KEGG sequence and metadata is downloaded in the GEM-PRO "sequences" folder.
    

- Usage:
        my_gempro.kegg_mapping_and_metadata(kegg_organism_code, custom_gene_mapping=None, force_rerun=False)


- Arguments:

    - *kegg_organism_code*
        - See the full list of organisms here: http://www.genome.jp/kegg/catalog/org_list.html
        - E. coli MG1655 is "eco"
        - M. tuberculosis is "mtu"

    - *custom_gene_mapping*
        - If the model gene IDs differ from the KEGG gene IDs, and you know the mapping, supply it as a dictionary here. 
        - An example would be for the *T. maritima* model, where the model IDs are formatted without an underscore ("TM0001") while in KEGG they have an underscore ("TM_0001").

    - *force_rerun*
        - If you want to force the rerun of mapping, set this to True.


- Creates attributes:
    - A summary of the metadata is available in the "df_kegg_metadata" attribute.
            my_gempro.df_kegg_metadata
            
    - Any gene IDs that are missing a mapping are reported in the "missing_kegg_mapping" attribute.
            my_gempro.missing_kegg_mapping
        
    
##### Mapping with UniProt

- Method:
    - You can try mapping your genes using the actual service here: http://www.uniprot.org/uploadlists/
    
    
- Files downloaded:
    - Per gene, the sequence and metadata is downloaded in the GEM-PRO "sequences" folder.
    

- Usage:
        my_gempro.uniprot_mapping_and_metadata(model_gene_source, custom_gene_mapping=None, force_rerun=False)
        
        
- Arguments:

    - *model_gene_source*
        - Here is a list of the gene IDs that can be mapped to UniProt IDs: http://www.uniprot.org/help/programmatic_access#id_mapping_examples
        - *E. coli* b-numbers are of the source "ENSEMBLGENOME_ID"
        - *M. tuberculosis* gene IDs match the source "TUBERCULIST_ID"
        
    - *custom_gene_mapping*
        - If you know the model gene ID to UniProt ID mapping, supply it as a dictionary here.
        
    - *force_rerun*
        - If you want to force the rerun of mapping, set this to True.
        
        
- Creates attributes:
    - A summary of the metadata is available in the "df_uniprot_metadata" attribute.
            my_gempro.df_uniprot_metadata
            
    - Any gene IDs that are missing a mapping are reported in the "missing_uniprot_mapping" attribute.
            my_gempro.missing_uniprot_mapping
        
            
##### Consolidating information

If you have mapped with both KEGG and UniProt mappers, then you can set a representative sequence for the gene using this function.

- Method:
    - Manual mappings override all existing mappings.
    - UniProt mappings override KEGG mappings except when KEGG mappings have PDBs associated with them and UniProt doesn't.


- Usage:
        my_gempro.set_representative_sequence()


- Creates attributes:
    - A summary of the mappings available in the "df_sequence_mapping" attribute.
            my_gempro.df_sequence_mapping
            
    - Any gene IDs that are missing a mapping are reported in the "missing_mapping" attribute.
            my_gempro.missing_mapping
        

---------------------------


### 2. I have the sequences for my organism

If you already have the amino acid sequences for each gene, simply set them as the representative sequence using the below function.

##### Setting the amino acid sequences

- Usage:
        my_gempro.manual_seq_mapping(gene_to_seq_dict)
        
        
- Arguments:

    - *gene_to_seq_dict*
        - Supply a dictionary mapping the model gene IDs to their amino acid sequences.
        - Example: 
                {'Rv1295': 'MFNGLNPTEDFVVLFEKDGNIYIADAPLSQRGVGKRSF'}

-------------------

### Gene annotations

Check out the gene annotations saved directly into the gene. 

- **These are COBRApy Gene objects! If you started with a SBML model, these are directly annotated within the model.**

- Usage:
        my_gempro.genes.get_by_id('Rv1295').annotation
        my_gempro.genes.get_by_id('Rv1295').annotation['sequence']

In [17]:
# KEGG mapping of gene ids
my_gempro.kegg_mapping_and_metadata(kegg_organism_code='mtu')
my_gempro.df_kegg_metadata.head(2)
my_gempro.missing_kegg_mapping

100%|██████████| 661/661 [00:02<00:00, 301.98it/s]
INFO:ssbio.pipeline.gempro:Created KEGG metadata dataframe. See the "df_kegg_metadata" attribute.


,gene,uniprot_acc,kegg_id,seq_len,pdbs,seq_file,metadata_file
0,Rv0417,P9WG73,mtu:Rv0417,252.0,NaN,mtu-Rv0417.faa,mtu-Rv0417.kegg
1,Rv2291,P9WHF5,mtu:Rv2291,284.0,NaN,mtu-Rv2291.faa,mtu-Rv2291.kegg


['Rv1755c', 'Rv2233', 'Rv0619', 'Rv0618', 'Rv2321c', 'Rv2322c']

In [19]:
# UniProt mapping of gene_id -> uniprot_id
my_gempro.uniprot_mapping_and_metadata(model_gene_source='TUBERCULIST_ID')
my_gempro.df_uniprot_metadata.head(2)

INFO:root:getUserAgent: Begin
INFO:root:getUserAgent: user_agent: EBI-Sample-Client/ (services.py; Python 3.5.2; Linux) Python-requests/2.11.1
INFO:root:getUserAgent: End
100%|██████████| 661/661 [00:00<00:00, 2422.65it/s]
INFO:ssbio.pipeline.gempro:Updated existing UniProt dataframe.


,gene,uniprot_acc,seq_len,seq_file,pdbs,gene_name,reviewed,kegg_id,refseq,ec_number,pfam,description,entry_version,seq_version,metadata_file
0,Rv0417,P9WG73,252.0,P9WG73.fasta,[],thiG,True,[mtu:Rv0417],"[NP_214931.1, NC_000962.3, WP_003916659.1, NZ_...",[2.8.1.10],NaN,[Thiazole synthase {ECO:0000255|HAMAP-Rule:MF_...,2016-10-05,2014-04-16,P9WG73.txt
1,Rv2291,P9WHF5,284.0,P9WHF5.fasta,[],sseB,True,[mtu:Rv2291],"[NP_216807.1, NC_000962.3, WP_003899253.1, NZ_...",[2.8.1.1],[PF00581],[Putative thiosulfate sulfurtransferase SseB],2016-10-05,2014-04-16,P9WHF5.txt


In [8]:
# Manually adding in UniProt mappings later
manual_uniprot = pd.read_csv(op.join(my_gempro.data_dir, '161019-gene_to_uniprot.in'))
manual_uniprot_dict = {}
for i,r in manual_uniprot.iterrows():
    manual_uniprot_dict[r[0]] = r[1]
print(manual_uniprot_dict)
my_gempro.manual_uniprot_mapping(manual_uniprot_dict)

{'Rv0619': 'Q79FY3', 'Rv2321c': 'P71891', 'Rv0618': 'Q79FY4', 'Rv2322c': 'P71890', 'Rv1755c': 'P9WIA9'}


INFO:ssbio.pipeline.gempro:Updated existing UniProt dataframe.


In [20]:
# Consolidate mappings sources
my_gempro.set_representative_sequence()
my_gempro.df_sequence_mapping.head(2)
my_gempro.missing_mapping

INFO:ssbio.pipeline.gempro:Created sequence mapping dataframe. See the "df_sequence_mapping" attribute.


,gene,uniprot_acc,kegg_id,pdbs,seq_len,seq_file,metadata_file
0,Rv0417,P9WG73,mtu:Rv0417,NaN,252.0,P9WG73.fasta,P9WG73.txt
1,Rv2291,P9WHF5,mtu:Rv2291,NaN,284.0,P9WHF5.fasta,P9WHF5.txt


['Rv2233']

In [21]:
# Looking at sequence information saved per gene
my_gempro.genes.get_by_id('Rv0417').annotation['sequence']

{'kegg': {'kegg_id': 'mtu:Rv0417',
  'metadata_file': 'mtu-Rv0417.kegg',
  'pdbs': [],
  'seq_file': 'mtu-Rv0417.faa',
  'seq_len': 252,
  'uniprot_acc': 'P9WG73'},
 'representative': {'kegg_id': ['mtu:Rv0417'],
  'metadata_file': 'P9WG73.txt',
  'pdbs': [],
  'seq_file': 'P9WG73.fasta',
  'seq_len': 252,
  'uniprot_acc': 'P9WG73'},
 'uniprot': {'P9WG73': {'description': ['Thiazole synthase {ECO:0000255|HAMAP-Rule:MF_00443}'],
   'ec_number': ['2.8.1.10'],
   'entry_version': '2016-10-05',
   'gene': 'Rv0417',
   'gene_name': 'thiG',
   'kegg_id': ['mtu:Rv0417'],
   'metadata_file': 'P9WG73.txt',
   'pdbs': [],
   'refseq': ['NP_214931.1', 'NC_000962.3', 'WP_003916659.1', 'NZ_KK339370.1'],
   'reviewed': True,
   'seq_file': 'P9WG73.fasta',
   'seq_len': 252,
   'seq_version': '2014-04-16',
   'uniprot_acc': 'P9WG73'}}}

## Mapping representative sequence to structure

There are two ways to map sequence to structure:

1. Use the UniProt ID and their automatic mappings to the PDB
2. BLAST the sequence to the PDB

As you can see, if you mapped UniProt IDs, then you can utilize an existing web service to map to PDBs. If not, you'll have to BLAST your sequence to the PDB. You can also run both for maximum coverage.

---------------------

### 1. Use the UniProt ID and their automatic mappings to the PDB


- Method:
    - Use the PDBe REST service to query for the best PDB structures for a UniProt ID.
    - Here is the ranking algorithm described by the PDB paper: https://nar.oxfordjournals.org/content/44/D1/D385.full
    - More information found here: https://www.ebi.ac.uk/pdbe/api/doc/sifts.html
    - Link used to retrieve results: https://www.ebi.ac.uk/pdbe/api/mappings/best_structures/:accession
    - The list of PDB structures mapping to a UniProt accession sorted by coverage of the protein and, if the same, resolution.


- Files downloaded:
    - Saves a .json file directly from the web request in the GEM-PRO "sequences" folder
    - No PDBs are downloaded yet
    
    
- Usage:
        my_gempro.map_uniprot_to_pdb(seq_ident_cutoff=0, force_rerun=False)
        
        
- Arguments:

    - *seq_ident_cutoff*
        - From 0 to 1
        - Provide the seq_ident_cutoff as a percentage to filter for structures with only a percent identity above the cutoff.
        - **Warning:** if you set the seq_ident_cutoff too high you risk filtering out PDBs that do match the sequence, but are just missing large portions of it.
        
    - *force_rerun*
        - If you want to force the rerun of mapping, set this to True.
        
        
- Creates attributes:

    - A summary of the rankings is available in the "df_pdb_ranking" attribute.
            my_gempro.df_pdb_ranking
    
        
---------------------------


### 2. BLAST the sequence to the PDB

This will BLAST the representative sequence against the entire PDB, and return significant hits. This will however return hits in other organisms, which may not be ideal.


- Method:
    - BLAST the representative sequence to the entire PDB.


- Files downloaded:
    - Saves a .xml file directly from the web request in the GEM-PRO "sequences" folder
    - No PDBs are downloaded yet
    
    
- Usage:
        my_gempro.blast_seqs_to_pdb(seq_ident_cutoff=0, evalue=0.0001, all_genes=False, force_rerun=False, display_link=False)
        
        
- Arguments:

    - *seq_ident_cutoff*
        - From 0 (default) to 1
        - Provide the seq_ident_cutoff as a percentage to filter for structures with only a percent identity above the cutoff.
        - **Warning:** if you set the seq_ident_cutoff too high you risk filtering out PDBs that do match the sequence, but are just missing large portions of it.
        
    - *evalue*
        - Cutoff for the E-value - filters for significant hits.
        - 0.001 is liberal, 0.0001 is stringent (default).
        
    - *all_genes*
        - Set to True if you want to BLAST all gene sequences
        - Set to False if you only want to BLAST genes without any PDBs (if map_uniprot_to_pdb was already run)
        
    - *force_rerun*
        - If you want to force the rerun of mapping, set this to True.
        
    - *display_link*
        - Display a link to the HTML results of the BLAST result per gene.
        
        
- Creates attributes:

    - A summary of the rankings is available in the "df_pdb_blast" attribute.
            my_gempro.df_pdb_blast
            

-------------------


### Gene annotations

Check out the gene annotations saved directly into the gene.

- Usage:
        my_gempro.genes.get_by_id('Rv1295').annotation['structure']['pdb']


In [22]:
# Mapping using the PDBe best_structures service
my_gempro.map_uniprot_to_pdb()

100%|██████████| 661/661 [00:04<00:00, 160.34it/s]
INFO:ssbio.pipeline.gempro:Completed UniProt -> best PDB mapping. See the "df_pdb_ranking" attribute.


In [23]:
# Mapping using BLAST
my_gempro.blast_seqs_to_pdb(seq_ident_cutoff=.99, all_genes=True)

 59%|█████▊    | 388/661 [00:01<00:00, 386.01it/s]INFO:ssbio.pipeline.gempro:Rv0548c: Adding 1 PDBs from BLAST results.
INFO:ssbio.pipeline.gempro:Rv2043c: Adding 1 PDBs from BLAST results.
INFO:ssbio.pipeline.gempro:Rv2445c: Adding 1 PDBs from BLAST results.
 66%|██████▋   | 439/661 [00:01<00:00, 415.37it/s]INFO:ssbio.pipeline.gempro:Rv0126: Adding 2 PDBs from BLAST results.
INFO:ssbio.pipeline.gempro:Rv2537c: Adding 2 PDBs from BLAST results.
INFO:ssbio.pipeline.gempro:Rv2753c: Adding 1 PDBs from BLAST results.
 88%|████████▊ | 579/661 [00:01<00:00, 351.44it/s]INFO:ssbio.pipeline.gempro:Rv1484: Adding 5 PDBs from BLAST results.
INFO:ssbio.pipeline.gempro:Rv0533c: Adding 1 PDBs from BLAST results.
INFO:ssbio.pipeline.gempro:Rv0859: Adding 1 PDBs from BLAST results.
100%|██████████| 661/661 [00:01<00:00, 347.42it/s]
INFO:ssbio.pipeline.gempro:Completed sequence --> PDB BLAST. See the "df_pdb_blast" attribute.


## Mapping representative sequence to structure

There are two ways to map sequence to structure:

1. Use the UniProt ID and their automatic mappings to the PDB
2. BLAST the sequence to the PDB

As you can see, if you mapped UniProt IDs, then you can utilize an existing web service to map to PDBs. If not, you'll have to BLAST your sequence to the PDB. You can also run both for maximum coverage.

---------------------

### 1. Use the UniProt ID and their automatic mappings to the PDB


- Method:
    - Use the PDBe REST service to query for the best PDB structures for a UniProt ID.
    - Here is the ranking algorithm described by the PDB paper: https://nar.oxfordjournals.org/content/44/D1/D385.full
    - More information found here: https://www.ebi.ac.uk/pdbe/api/doc/sifts.html
    - Link used to retrieve results: https://www.ebi.ac.uk/pdbe/api/mappings/best_structures/:accession
    - The list of PDB structures mapping to a UniProt accession sorted by coverage of the protein and, if the same, resolution.


- Files downloaded:
    - Saves a .json file directly from the web request in the GEM-PRO "sequences" folder
    - No PDBs are downloaded yet
    
    
- Usage:
        my_gempro.map_uniprot_to_pdb(seq_ident_cutoff=0, force_rerun=False)
        
        
- Arguments:

    - *seq_ident_cutoff*
        - From 0 to 1
        - Provide the seq_ident_cutoff as a percentage to filter for structures with only a percent identity above the cutoff.
        - **Warning:** if you set the seq_ident_cutoff too high you risk filtering out PDBs that do match the sequence, but are just missing large portions of it.
        
    - *force_rerun*
        - If you want to force the rerun of mapping, set this to True.
        
        
- Creates attributes:

    - A summary of the rankings is available in the "df_pdb_ranking" attribute.
            my_gempro.df_pdb_ranking
    

In [12]:
old_gene_to_homology = pd.read_csv('~/projects/iNJ661/data_frames/DF_01_RXN_GENE_UNIPROT_PDB.csv')
old_gene_to_homology = old_gene_to_homology[['m_gene','u_uniprot_acc']].drop_duplicates().reset_index(drop=True)
old_gene_to_homology = old_gene_to_homology[pd.notnull(old_gene_to_homology.u_uniprot_acc)]
old_gene_to_homology.to_csv(op.join(my_gempro.data_dir,'161031-old_gene_to_uniprot_mapping.csv'))
gene_to_uniprot = old_gene_to_homology.set_index('m_gene').to_dict()['u_uniprot_acc']

In [13]:
my_gempro.get_itasser_models(homology_raw_dir='/home/nathan/projects_unsynced/iNJ661/structure_files/homology_models_prep', custom_itasser_name_mapping=gene_to_uniprot)

100%|██████████| 661/661 [00:00<00:00, 5759.53it/s]
ssbio.gempro.pipeline - INFO - Completed copying of I-TASSER models to GEM-PRO directory. See the "df_itasser" attribute.


In [47]:
my_gempro.set_representative_structure()

100%|██████████| 661/661 [00:00<00:00, 340510.31it/s]


In [49]:
my_gempro.genes[33].annotation['structure']

{'homology': {'Rv1731': {'c_score': -0.22,
   'difficulty': 'easy',
   'model_date': '2016-01-08',
   'model_file': 'Rv1731.pdb',
   'rmsd': 7.9,
   'rmsd_err': 4.4,
   'seq_coverage': 1,
   'tm_score': 0.69,
   'tm_score_err': 0.12,
   'top_template_chain': 'A',
   'top_template_pdb': '4h73'}},
 'pdb': OrderedDict(),
 'representative': {'clean_pdb_file': None,
  'original_pdb_file': None,
  'seq_coverage': 0,
  'structure_id': None}}

In [45]:
my_gempro.genes[3].annotation['structure']['pdb']

OrderedDict([('2d1f_A',
              {'experimental_method': 'X-ray diffraction',
               'pdb_chain_id': 'A',
               'pdb_end': 360,
               'pdb_id': '2d1f',
               'pdb_start': 1,
               'rank': 1,
               'release_date': '2006-09-05',
               'resolution': 2.5,
               'seq_coverage': 1,
               'taxonomy_id': 1773,
               'uniprot_acc': 'P9WG59',
               'unp_end': 360,
               'unp_start': 1}),
             ('2d1f_B',
              {'experimental_method': 'X-ray diffraction',
               'pdb_chain_id': 'B',
               'pdb_end': 360,
               'pdb_id': '2d1f',
               'pdb_start': 1,
               'rank': 2,
               'release_date': '2006-09-05',
               'resolution': 2.5,
               'seq_coverage': 1,
               'taxonomy_id': 1773,
               'uniprot_acc': 'P9WG59',
               'unp_end': 360,
               'unp_start': 1})])

In [21]:
import time
import os


'20160104'

In [18]:
d = my_gempro.genes.get_by_id('Rv2043c').annotation['structure']['homology']

In [40]:
d['rvvv'] = {'c_score': 26,
  'difficulty': 'easy',
  'model_date': '2016-12-29',
  'model_file': 'Rv2043c.pdb',
  'rmsd': 1.7,
  'rmsd_err': 1.5,
  'seq_coverage': 1,
  'tm_score': 0,
  'tm_score_err': 0.05,
  'top_template_chain': 'A',
  'top_template_pdb': '3gbc'}

In [41]:
d

{'Rv2043c': {'c_score': 1.86,
  'difficulty': 'easy',
  'model_date': '2015-12-29',
  'model_file': 'Rv2043c.pdb',
  'rmsd': 1.7,
  'rmsd_err': 1.5,
  'seq_coverage': 1,
  'tm_score': 0.98,
  'tm_score_err': 0.05,
  'top_template_chain': 'A',
  'top_template_pdb': '3gbc'},
 'rvvv': {'c_score': 26,
  'difficulty': 'easy',
  'model_date': '2016-12-29',
  'model_file': 'Rv2043c.pdb',
  'rmsd': 1.7,
  'rmsd_err': 1.5,
  'seq_coverage': 1,
  'tm_score': 0,
  'tm_score_err': 0.05,
  'top_template_chain': 'A',
  'top_template_pdb': '3gbc'}}

In [42]:
sorted(d, key=lambda x: d[x]['model_date'], reverse=True)

['rvvv', 'Rv2043c']